In [ ]:
! pip install sentence_transformers keybert bertopic


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/671.8 kB ? eta -:--:--
   --------------------------------------- 671.8/671.8 kB 13.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   -------------- ------------------------- 3.7/9.9 MB 18.2 MB/s eta 0:00:01
   ------------------------------ --------- 7.6/9.9 MB 18.1 MB/s eta 0:00:01
   ---------------------------------------- 9.9/9.9 MB 17.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   -------------------- ------------------- 4.2/8.0 MB 21.0 MB/s eta 0:00:01
   ---------------------------------------  7.9/8.0 MB 20.3 MB/s eta 0:00:01
   ---------------------------------------- 8.0/8.0 MB 18.4 MB/s eta 0:00:00

   ---------------------------------------- 0/9 [narwhals]
   ---------------------------------------- 0/9 [narwhals]
   ---------------------------------------- 0/9 [narwhals]
   ---------------------------------------- 0/9 [narwhals]
   --------

In [4]:
import pandas as pd

df = pd.read_csv("data\\Emotion NLP\\journal_texts\\journal_texts.csv")   # or read_excel
texts = (
  df["text"]
  .dropna()
  .astype(str)
  .str.strip()
  .tolist()
)

texts = [t for t in texts if len(t.split()) >= 5]


In [6]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\carli\.pyenv\pyenv-win\versions\3.12.3\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\carli\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## KeyBERT

In [80]:
from keybert import KeyBERT

kw_model = KeyBERT(model=embedding_model)

keybert_results = []
for text in texts:
  keywords = kw_model.extract_keywords(
    text,
    keyphrase_ngram_range=(1, 2),
    stop_words="english",
    top_n=5
  )
  keybert_results.append([kw for kw, score in keywords])


In [81]:
df = df.iloc[:len(keybert_results)]
df["keybert_keywords"] = keybert_results


In [95]:
df['keybert_keywords'][0]

['felt heavier', 'heavier expected', 'heavier', 'today felt', 'conversation']

In [121]:
import spacy

# Load English model
nlp = spacy.load("en_core_web_sm")

def pick_best_noun(keyphrases):
  for phrase in keyphrases:
    doc = nlp(phrase)
    if len(doc) == 1 and doc[0].pos_ in ["NOUN", "PROPN"]:
      return phrase
    elif len(doc) == 1 and doc[0].pos_ == "VERB":
      return phrase
  # fallback: first phrase
  return keyphrases[0] if keyphrases else None



In [122]:
df["keybert_best"] = df["keybert_keywords"].apply(pick_best_noun)


In [123]:
df["keybert_best"]

0       conversation
1          breathing
2           thoughts
3         frustrated
4               calm
5           thinking
6         felt tired
7          defensive
8           feelings
9            writing
10      interactions
11      sit emotions
12      restlessness
13      disconnected
14       reassurance
15              mood
16         felt flat
17    comparing feel
18            relief
19           feeling
Name: keybert_best, dtype: object

## BERTopic

In [17]:
from bertopic import BERTopic

topic_model = BERTopic(
  embedding_model=embedding_model,
  calculate_probabilities=True
  #verbose=False
)

topics, probs = topic_model.fit_transform(texts)

df["bertopic_topic"] = topics


In [19]:
df["bertopic_topic"]

0    -1
1    -1
2    -1
3    -1
4    -1
5    -1
6    -1
7    -1
8    -1
9    -1
10   -1
11   -1
12   -1
13   -1
14   -1
15   -1
16   -1
17   -1
18   -1
19   -1
Name: bertopic_topic, dtype: int64

In [14]:
topic_info = topic_model.get_topic_info()

In [15]:
topic_model.get_topic(0)


False

## Similarity Themes

In [29]:
THEMES = {
  "Anxiety and fear": 
    "worry fear nervous anticipation threat uncertainty",
  
  "Low mood and sadness": 
    "sadness emptiness heaviness hopelessness low energy",

  "Stress and overload": 
    "stress pressure overwhelm responsibility tension",

  "Anger and frustration": 
    "anger irritation resentment annoyance conflict",

  "Self-criticism and doubt": 
    "self criticism inadequacy shame doubt self blame",

  "Interpersonal connection": 
    "relationships connection belonging closeness support",

  "Isolation and withdrawal": 
    "loneliness disconnection avoidance withdrawal",

  "Physical tension and arousal": 
    "body tension tight chest breathing fatigue",

  "Hope and motivation": 
    "hope motivation optimism goals meaning"
}


In [30]:
import numpy as np

model = SentenceTransformer("all-MiniLM-L6-v2")

In [31]:
theme_names = list(THEMES.keys())
theme_texts = list(THEMES.values())

theme_embeddings = model.encode(
  theme_texts,
  normalize_embeddings=True
)


In [32]:
from sklearn.metrics.pairwise import cosine_similarity

def assign_themes(text, top_k=3):
  text_embedding = model.encode(
    text,
    normalize_embeddings=True
  ).reshape(1, -1)

  sims = cosine_similarity(text_embedding, theme_embeddings)[0]

  ranked = sorted(
    zip(theme_names, sims),
    key=lambda x: x[1],
    reverse=True
  )

  return ranked[:top_k]


In [38]:
df["similarity_themes"] = df["text"].apply(
  lambda t: assign_themes(t, top_k=3)
)

df["similarity_theme"] = df["similarity_themes"].apply(
  lambda x: x[0][0] if len(x) > 0 else None
)

In [42]:
df["similarity_theme"]

0     Physical tension and arousal
1     Physical tension and arousal
2             Low mood and sadness
3              Stress and overload
4                 Anxiety and fear
5                 Anxiety and fear
6     Physical tension and arousal
7            Anger and frustration
8             Low mood and sadness
9             Low mood and sadness
10                Anxiety and fear
11           Anger and frustration
12            Low mood and sadness
13        Isolation and withdrawal
14                Anxiety and fear
15           Anger and frustration
16            Low mood and sadness
17            Low mood and sadness
18        Isolation and withdrawal
19                Anxiety and fear
Name: similarity_theme, dtype: object

## Comparable Outputs

In [ ]:
from collections import Counter
import itertools

all_keywords = Counter(
  itertools.chain.from_iterable(df["keybert_keywords"])
)

top_keybert_terms = all_keywords.most_common(20)